## Практическая работа №3

## 1. Решение задачи (Вариант №1)

### 1.1 Подготовка датафрейма

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn import model_selection, preprocessing
from sklearn.preprocessing import LabelEncoder

### 1.2 Исходный датафрейм
Предварительно в полях 'Type 1' и 'Type 2' заменяем NaN на строковое значение 'nan', так как отсутствие признака в одном из полей лишь уточняет принадлежность покемона только к одному из типов, а не делает непригодным данную строку датасета.

In [2]:
data = pd.read_csv('pok.csv')
data = data.fillna('nan')
data = data.dropna()
data

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,nan,309,39,52,43,60,50,65,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True
796,719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True
798,720,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,80,6,True


In [3]:
categorial_columns = data["Type 1"] + " " + data["Type 2"]
data = data.drop("Type 1", axis = 1)
data = data.drop("Type 2", axis = 1)

categorial_columns_res = []
for elem in categorial_columns:
    categorial_columns_res.append(sorted(elem.split()))
    
result = []
for i in range(len(categorial_columns_res)):
            result.append(categorial_columns_res[i])

result_strings = []
for i in range(len(result)):
    result_strings.append(result[i][0] + " " + result[i][1])
    
label_encoder = LabelEncoder()
categorial_columns = label_encoder.fit_transform(result_strings)
data["Categorial"] = categorial_columns
print("Число нейронов в выходном слое: " + str(len(set(categorial_columns))))

Число нейронов в выходном слое: 133


### 1.3 Конечный датафрейм 
(Преобразовали категориальные признаки 'Type 1' и 'Type 2' (объединили 'Type 1' и 'Type 2' и представили как один категориальный признак) к числовым и поместили их в новый стобец 'Categorial')

In [4]:
data

,#,Name,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary,Categorial
0,1,Bulbasaur,318,45,49,49,65,65,45,1,False,100
1,2,Ivysaur,405,60,62,63,80,80,60,1,False,100
2,3,Venusaur,525,80,82,83,100,100,80,1,False,100
3,3,VenusaurMega Venusaur,625,80,100,123,122,120,80,1,False,100
4,4,Charmander,309,39,52,43,60,50,65,1,False,77
...,...,...,...,...,...,...,...,...,...,...,...,...
795,719,Diancie,600,50,100,150,100,150,50,6,True,55
796,719,DiancieMega Diancie,700,50,160,110,160,110,110,6,True,55
797,720,HoopaHoopa Confined,600,80,110,60,150,130,70,6,True,93
798,720,HoopaHoopa Unbound,680,80,160,60,170,130,80,6,True,21


### 1.4 Построение модели

In [5]:
data_x = np.array(data.drop(columns = ['Name', 'Total', 'Generation', 'Legendary', '#', 'Categorial']))
data_y = np.array(data['Categorial'], dtype = 'float')
data_y.shape = (len(data_y), 1)
pd.DataFrame(data_x)

,0,1,2,3,4,5
0,45,49,49,65,65,45
1,60,62,63,80,80,60
2,80,82,83,100,100,80
3,80,100,123,122,120,80
4,39,52,43,60,50,65
...,...,...,...,...,...,...
795,50,100,150,100,150,50
796,50,160,110,160,110,110
797,80,110,60,150,130,70
798,80,160,60,170,130,80


In [143]:
data_x = preprocessing.scale(data_x)
x_train, x_test, y_train, y_test = model_selection.train_test_split(data_x, data_y, train_size = 0.7)
x_f_train = x_train
x_f_test = x_test

y_f_train = y_train
y_f_test = y_test

In [147]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(30, activation = tf.nn.relu))
model.add(tf.keras.layers.Dense(50, activation = tf.nn.relu))
model.add(tf.keras.layers.Dense(50, activation = tf.nn.relu))
model.add(tf.keras.layers.Dense(133, activation = tf.nn.softmax))

model.compile(optimizer = "adam", 
              loss = 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

### 1.5 Результат

In [151]:
model.fit(x_f_train, y_f_train, epochs = 200)

Epoch 1/200
18/18 [==============================] - 0s 2ms/step - loss: 0.1967 - accuracy: 0.9500
Epoch 2/200
18/18 [==============================] - 0s 2ms/step - loss: 0.2036 - accuracy: 0.9482
Epoch 3/200
18/18 [==============================] - 0s 2ms/step - loss: 0.2040 - accuracy: 0.9482
Epoch 4/200
18/18 [==============================] - 0s 2ms/step - loss: 0.1882 - accuracy: 0.9625
Epoch 5/200
18/18 [==============================] - 0s 2ms/step - loss: 0.1811 - accuracy: 0.9625
Epoch 6/200
18/18 [==============================] - 0s 2ms/step - loss: 0.1978 - accuracy: 0.9500
Epoch 7/200
18/18 [==============================] - 0s 2ms/step - loss: 0.1963 - accuracy: 0.9571
Epoch 8/200
18/18 [==============================] - 0s 2ms/step - loss: 0.1954 - accuracy: 0.9375
Epoch 9/200
18/18 [==============================] - 0s 2ms/step - loss: 0.1928 - accuracy: 0.9446
Epoch 10/200
18/18 [==============================] - 0s 2ms/step - loss: 0.1960 - accuracy: 0.9536
Epoch 11/

In [149]:
score, acc = model.evaluate(x_f_test, y_f_test)

8/8 [==============================] - 0s 2ms/step - loss: 13.7655 - accuracy: 0.1583


## 2. Решение задачи (Вариант №2)

In [21]:
import tensorflow as tf
from sklearn import preprocessing, model_selection
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from keras import initializers

### 2.1 Подготовка датафрейма

In [22]:
data = pd.read_csv('pok.csv')
print(set(data['Type 1']), len(set(data['Type 1'])))
print(set(data['Type 2']), len(set(data['Type 2'])))

{'Fighting', 'Rock', 'Grass', 'Ghost', 'Dark', 'Water', 'Steel', 'Flying', 'Electric', 'Poison', 'Ice', 'Dragon', 'Normal', 'Fairy', 'Bug', 'Psychic', 'Fire', 'Ground'} 18
{nan, 'Fighting', 'Water', 'Electric', 'Poison', 'Fairy', 'Normal', 'Ground', 'Grass', 'Dark', 'Ice', 'Flying', 'Ghost', 'Steel', 'Dragon', 'Bug', 'Psychic', 'Fire', 'Rock'} 19


### 2.2 Исходный датафрейм

In [23]:
data[['Type 1', 'Type 2']] = df[['Type 1', 'Type 2']].fillna('nan')
data = data.dropna()
data

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,nan,309,39,52,43,60,50,65,1,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,719,Diancie,Rock,Fairy,600,50,100,150,100,150,50,6,True
796,719,DiancieMega Diancie,Rock,Fairy,700,50,160,110,160,110,110,6,True
797,720,HoopaHoopa Confined,Psychic,Ghost,600,80,110,60,150,130,70,6,True
798,720,HoopaHoopa Unbound,Psychic,Dark,680,80,160,60,170,130,80,6,True


### 2.3 Выделение признаков
В данном решении признаки 'Type 1' и 'Type 2' выделяются независимо друг от друга.

In [24]:
x = np.array(data.drop(columns = ['#','Name', 'Type 1', 'Type 2', 'Generation', 'Legendary'], axis = 1))
y = np.array(data[['Type 1', 'Type 2']])

a = 0
for i in range(len(y)):
    y[i] = sorted(y[i])
    a += 1
    if a > 10:
        break

labelencoder = LabelEncoder()
a = labelencoder.fit_transform(y[:,0])
b = labelencoder.fit_transform(y[:,1])

y_code = np.array(list(zip(a,b)))
print(y_code[:30,:])

[[ 9 13]
 [ 9 13]
 [ 9 13]
 [ 9 13]
 [ 6 18]
 [ 6 18]
 [ 6  7]
 [ 2  6]
 [ 6  7]
 [17 18]
 [17 18]
 [17 18]
 [17 18]
 [ 0 18]
 [ 0 18]
 [ 0  7]
 [ 0 13]
 [ 0 13]
 [ 0 13]
 [ 0 13]
 [12  7]
 [12  7]
 [12  7]
 [12  7]
 [12 18]
 [12 18]
 [12  7]
 [12  7]
 [13 18]
 [13 18]]


### 2.4 Построение модели

In [25]:
x_train, x_test, y_train, y_test = model_selection.train_test_split(x, y_code, test_size = 0.7)
x_f_train = preprocessing.scale(x_train)
x_f_test = preprocessing.scale(x_test)

y_f_train = y_train
y_f_test = y_test

In [26]:
X = x_f_train
Y = y_f_train

inputs = tf.keras.Input(shape = (x_f_train.shape[1],), name = 'input')
x = tf.keras.layers.Dense(100, activation = 'relu')(inputs)
z = tf.keras.layers.Dense(300, activation = 'relu')(x)
y = tf.keras.layers.Dense(100, activation = 'relu')(z)
output1 = tf.keras.layers.Dense(19, activation = 'softmax' ,name = 'first_out')(y)
output2 = tf.keras.layers.Dense(19, activation = 'softmax', name = 'second_out')(y)

model1 = tf.keras.Model(inputs = inputs, outputs = [output1, output2])


model1.compile(loss= {
                      "first_out": tf.keras.losses.SparseCategoricalCrossentropy(from_logits = False),
                      "second_out": tf.keras.losses.SparseCategoricalCrossentropy(from_logits = False)
                     },
               optimizer = 'SGD',
               metrics = 'sparse_categorical_accuracy')

### 2.5 Результат

In [27]:
model1.fit(X, {"first_out": Y[:,0], "second_out": Y[:,1]}, epochs = 150)

Epoch 1/150
8/8 [==============================] - 1s 4ms/step - loss: 5.8842 - first_out_loss: 2.9435 - second_out_loss: 2.9408 - first_out_sparse_categorical_accuracy: 0.0458 - second_out_sparse_categorical_accuracy: 0.0708
Epoch 2/150
8/8 [==============================] - 0s 3ms/step - loss: 5.7959 - first_out_loss: 2.9280 - second_out_loss: 2.8679 - first_out_sparse_categorical_accuracy: 0.0667 - second_out_sparse_categorical_accuracy: 0.3125
Epoch 3/150
8/8 [==============================] - 0s 3ms/step - loss: 5.7101 - first_out_loss: 2.9128 - second_out_loss: 2.7973 - first_out_sparse_categorical_accuracy: 0.1125 - second_out_sparse_categorical_accuracy: 0.4625
Epoch 4/150
8/8 [==============================] - 0s 2ms/step - loss: 5.6275 - first_out_loss: 2.8982 - second_out_loss: 2.7293 - first_out_sparse_categorical_accuracy: 0.1500 - second_out_sparse_categorical_accuracy: 0.4958
Epoch 5/150
8/8 [==============================] - 0s 2ms/step - loss: 5.5454 - first_out_loss: 

In [28]:
model1.evaluate(x_f_test, {"first_out": y_f_test[:,0], "second_out": y_f_test[:,1]})

18/18 [==============================] - 0s 2ms/step - loss: 4.6203 - first_out_loss: 2.5024 - second_out_loss: 2.1179 - first_out_sparse_categorical_accuracy: 0.2143 - second_out_sparse_categorical_accuracy: 0.4554


[4.620321750640869,
 2.5024421215057373,
 2.1178789138793945,
 0.2142857164144516,
 0.4553571343421936]

In [31]:
res = np.array(model1.predict(x_f_test))
pred = np.zeros((2,560))
for i in range(len(res)):
    for j in range(len(res[i])):
        pred[i][j] = np.argmax(res[i,j])

pred = pred.transpose()
y_f_test = np.array(y_f_test)
count = 0
for i in range(len(pred)):
    if (pred[i] == y_f_test[i]).all():
        count = count + 1
        
print("Total accuracy:", count / len(y_f_test))

18/18 [==============================] - 0s 2ms/step
Total accuracy: 0.11071428571428571
